In [1]:
import arrow
from datetime import datetime
from apps.patterns.git import repo_at
from apps.patterns.models import OcurrenceReport
from apps.patterns.analyzer import analyze_repo_with_pattern

def to_arrow(in_datetime: datetime, tz: str = "America/Santiago") -> arrow.Arrow:
    return arrow.Arrow(
        year=in_datetime.year,
        month=in_datetime.month,
        day=in_datetime.day,
        hour=in_datetime.hour,
        minute=in_datetime.minute,
        second=in_datetime.second,
        tzinfo=tz
    )


def get_end_of_day(arrow_date, days_ago=1, tz="America/Santiago"):
    return arrow_date.shift(days=-days_ago).replace(hour=23, minute=59, second=59, tzinfo=tz)


now = arrow.now()
max_datetime = arrow.Arrow(year=2020, month=6, day=15, hour=0, mimute=0, second=0)
days_left_to_analyze = (now - max_datetime).days
last_of_days_commits = []
with repo_at(commit=None, force_delete=False, always_pull=True) as repo:
    current_day = 1
    skip_commits = 0
    max_page = 1000
    commits = repo.iter_commits("devel", max_count=max_page, skip=skip_commits)
    while current_day != days_left_to_analyze:
        critic_moment = get_end_of_day(now, days_ago=current_day)
        
        last_day_commit_found = False
        while not last_day_commit_found:
            try:
                commit = next(commits)
            except StopIteration:
                skip_commits += max_page
                commits = repo.iter_commits("devel", max_count=max_page, skip=skip_commits)
                commit = next(commits)
                
            commit_datetime = to_arrow(commit.committed_datetime)
            if commit_datetime < critic_moment:
                last_of_days_commits.append(commit)
                last_day_commit_found = True
                
        current_day += 1
        
for commit in last_of_days_commits:
    print(f"Analyzing commit {commit.hexsha}")
    with repo_at(commit=commit.hexsha, force_delete=False) as repo:
        hits = analyze_repo_with_pattern(pattern="apps.catalog.models")
        payload = dict(
            ocurrences=hits,
            commit=commit.hexsha,
            commited_epoch=to_arrow(commit.committed_datetime).timestamp,
            last_of_day=True
        )
        OcurrenceReport.objects.create_report(payload)

Analyzing commit fbdd23af87e4184fa7221228025327851e8ea254
Analyzing commit 9fe78dcf8834e1bc987f33b1ed079abf76a2919c
Analyzing commit 6ceec2ba0e8afb1d7a78bf73993cb6df62cb9431
Analyzing commit 7f56e56bc069a554da44447531cd9eec5329b368
Analyzing commit ca6dbda8e95985afac81890de6f03f643541e529
Analyzing commit cdec7b344a44b0d16e9d0dab204d27850c405801
Analyzing commit 6a0a0a6a809cdf969e48584bfccaf850b3a05ecb
Analyzing commit cfe593543f84e99df977abc434474687927bfdd2
Analyzing commit 92e387b5fb21f82d7aedf999ccc8a2dba4e9b271
Analyzing commit 23747cb61df8d828fcf403310a3e28806c78ef5c
Analyzing commit 5fccb8c3961dfb9ce6ffa040ce3768175960874b
Analyzing commit 4861b26aa2e142f00670e1225af9f9c19c3f379a
Analyzing commit 89926ef50f5e46780d000005a615f4ae45dd32f1
Analyzing commit 6d78fb3f593a59341828deb3165c9f1fa4a96e65
Analyzing commit 1fc51400777cda7608b1dfca8ad0d2bf20ab1aa2
Analyzing commit 15e0407494f072cad2a8176b22e64a5acd66e0d8
Analyzing commit 09f0acd230877749e17ab629809ff2348f4c75cf
Analyzing comm

In [8]:
import arrow
all_reports = OcurrenceReport.objects.filter(last_of_day=True).order_by('commited_epoch').values_list('_total', 'commited_epoch')
for tot, epoch in all_reports:
    print(arrow.get(epoch).format('YYYY-MM-DD'))
for tot, epoch in all_reports:
    print(tot)

2020-06-17
2020-06-18
2020-06-19
2020-06-20
2020-06-21
2020-06-22
2020-06-23
2020-06-24
2020-06-25
2020-06-26
2020-06-27
2020-06-28
2020-06-29
2020-06-30
2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10
2020-07-11
2020-07-12
2020-07-13
2020-07-14
2020-07-15
2020-07-16
2020-07-17
2020-07-18
2020-07-18
2020-07-19
2020-07-21
2020-07-22
2020-07-23
2020-07-24
2020-07-25
2020-07-26
2020-07-27
2020-07-28
2020-07-29
2020-07-30
2020-07-31
2020-08-01
2020-08-02
2020-08-03
2020-08-04
2020-08-05
2020-08-06
2020-08-07
2020-08-08
2020-08-08
2020-08-10
2020-08-11
2020-08-12
2020-08-13
2020-08-14
2020-08-15
2020-08-15
2020-08-17
2020-08-18
2020-08-19
2020-08-20
2020-08-21
2020-08-22
2020-08-23
2020-08-24
2020-08-25
2020-08-26
2020-08-27
2020-08-28
2020-08-29
2020-08-29
2020-08-31
2020-09-01
2020-09-02
2020-09-02
2020-09-03
2020-09-03
2020-09-04
2020-09-04
2020-09-05
2020-09-05
2020-09-05
2020-09-05
2020-09-07
2020-09-07
2020-09-08
2020-09-08